In [27]:
import sys
import cv2
import numpy as np

In [28]:
img1_source = 'cv22_lab2_material/part3 - ImageStitching/1.jpg'
img2_source = 'cv22_lab2_material/part3 - ImageStitching/2.jpg'
img3_source = 'cv22_lab2_material/part3 - ImageStitching/3.jpg'
img4_source = 'cv22_lab2_material/part3 - ImageStitching/4.jpg'
img5_source = 'cv22_lab2_material/part3 - ImageStitching/5.jpg'
img6_source = 'cv22_lab2_material/part3 - ImageStitching/6.jpg'

In [29]:
def stitchImages(img11, img22, result_img):
    # Load our images
    img1 = cv2.imread(img11)
    img2 = cv2.imread(img22)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    '''
    cv2.imshow('img1_gray.jpg',img1_gray)
    cv2.imshow('img2_gray.jpg',img2_gray)
    '''
    # Create our ORB detector and detect keypoints and descriptors
    orb = cv2.ORB_create(nfeatures=2000)

    # Find the key points and descriptors with ORB
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
    '''
    cv2.imshow('img1_keypoints.jpg',cv2.drawKeypoints(img1, keypoints1, None, (255, 0, 255)))
    cv2.imshow('img2_keypoints.jpg',cv2.drawKeypoints(img2, keypoints2, None, (255, 0, 255)))   
    '''
    # Create a BFMatcher object.
    # It will find all of the matching keypoints on two images
    bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)

    # Find matching points
    matches = bf.knnMatch(descriptors1, descriptors2,k=2)          
    '''
    print(keypoints1[0].pt)
    print(keypoints1[0].size)
    print("Descriptor of the first keypoint: ")
    print(descriptors1[0])
    '''
           
    all_matches = []
    for m, n in matches:
        all_matches.append(m)

    #img3 = draw_matches(img1_gray, keypoints1, img2_gray, keypoints2, all_matches[:30])
    draw_params = dict(matchColor=(0,255,0), singlePointColor=None, flags=2)
    img3 = cv2.drawMatches(img1_gray, keypoints1, img2_gray, keypoints2, all_matches[:30], None, **draw_params)
    '''
    cv2.imshow('drawMatches.jpg',img3)   
    '''
    
    # Finding the best matches
    good = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)
    '''    
    cv2.imshow('img1_drawKeypoints.jpg',cv2.drawKeypoints(img1, [keypoints1[m.queryIdx] for m in good], None, (255, 0, 255)))   
    cv2.imshow('img1_drawKeypoints.jpg',cv2.drawKeypoints(img2, [keypoints2[m.trainIdx] for m in good], None, (255, 0, 255))) 
    '''

    def warpImages(img1, img2, H):
        rows1, cols1 = img1.shape[:2]
        rows2, cols2 = img2.shape[:2]

        list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
        temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)

        # When we have established a homography we need to warp perspective
        # Change field of view
        list_of_points_2 = cv2.perspectiveTransform(temp_points, H)

        list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)

        [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
        [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)
  
        translation_dist = [-x_min,-y_min]
  
        H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

        output_img = cv2.warpPerspective(img2, H_translation.dot(H), (x_max-x_min, y_max-y_min))
        output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1

        return output_img
    
    MIN_MATCH_COUNT = 10

    if len(good) > MIN_MATCH_COUNT:
        # Convert keypoints to an argument for findHomography
        src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        # Establish a homography
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    
        result = warpImages(img2, img1, M)
        
        def trim(frame):
            if not np.sum(frame[0]):
                return trim(frame[1:])
            if not np.sum(frame[-1]):
                return trim(frame[:-2])
            if not np.sum(frame[:,0]):
                return trim(frame[:,1:])
            if not np.sum(frame[:,-1]):
                return trim(frame[:,:-2])
            return frame
        
        cv2.imshow('result.jpg',trim(result))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
    cv2.imwrite(result_img, trim(result))

In [30]:
img12='1_2.jpg'
stitchImages(img1_source,img2_source,img12)

In [31]:
img123='1_2_3.jpg'
stitchImages(img12,img3_source,img123)

In [32]:
img45='4_5.jpg'
stitchImages(img4_source,img5_source,img45)

In [33]:
img456='4_5_6.jpg'
stitchImages(img45,img6_source,img456)

In [34]:
end='final.jpg'
stitchImages(img123,img456,end)